In [1]:
from pathlib import Path
import pandas as pd
%matplotlib inline

In [2]:
df = pd.read_csv('df.csv')
df_contexts = pd.read_csv('contexts.csv', index_col='id')

In [3]:
df['source'].fillna('testing', inplace=True)
df = df[~df['contexts'].isnull()]
len(df)

45122

In [4]:
df.head()

,participant,source,cs,is_filler,word,group,contexts
0,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,49,True,длинный,no,331 332 333 334 335
1,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,24,False,беззаботный,close_far_meton,152 154 158
2,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,24,False,беззаботный,close_far_meton,159 155 151
3,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,24,False,беззаботный,close_far_meton,153 156 157
4,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,43,False,смелый,close_close_meton,297 295


In [5]:
df_contexts.head()

,cs,derivation,text
id,,,
1,1,metonymy,У него были маленькие злые глазки.
9,2,homonym1,Повышение по службе дало надежды на карьерный ...
18,3,literal,В этом году английский у нас преподает нервна...
25,4,homonym1,Наша задача ― посеять на лугах и пастбищах нов...
32,5,literal,"В ходе расследования выяснилось, что водитель ..."


In [6]:
df_fillers = df[df['is_filler']].copy()
df_fillers.head()

,participant,source,cs,is_filler,word,group,contexts
0,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,49,True,длинный,no,331 332 333 334 335
10,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,48,True,круглый,no,325 326 327 328 329 330
19,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,44,True,строевой,no,300 301 302 303 304 305 306
24,1a8f0842-6a05-4a24-9235-437b6c35ef73,testing,48,True,круглый,no,325 326 327 328 329 330
25,1a8f0842-6a05-4a24-9235-437b6c35ef73,testing,44,True,строевой,no,300 301 302 303 304 305 306


In [7]:
df_fillers[
    (df_fillers['participant'] == '1a8f0842-6a05-4a24-9235-437b6c35ef73')
    & (df_fillers['cs'] == 46)
]

,participant,source,cs,is_filler,word,group,contexts
27,1a8f0842-6a05-4a24-9235-437b6c35ef73,testing,46,True,половой,no,312 313 314
28,1a8f0842-6a05-4a24-9235-437b6c35ef73,testing,46,True,половой,no,315 316 317 318 319


In [8]:
df_fillers['derivations'] = df_fillers['contexts'].apply(
    lambda cs: {df_contexts.loc[int(c_id)]['derivation'] for c_id in cs.split()})

In [9]:
df_fillers['derivations'][:10]

0               {monosemy}
10              {monosemy}
19    {homonym1, homonym2}
24              {monosemy}
25    {homonym1, homonym2}
27              {homonym1}
28    {homonym1, homonym2}
35              {monosemy}
42              {monosemy}
43              {monosemy}
Name: derivations, dtype: object

In [10]:
df_fillers['homogeneous_group'] = df_fillers['derivations'].apply(lambda s: len(s) == 1)

In [11]:
df_fillers['homogeneous_group'].mean()

0.9608126410835215

In [12]:
all_homogeneous = df_fillers.groupby(['participant', 'source'])['homogeneous_group'].min().reset_index()

In [13]:
all_homogeneous.groupby(['source'])['homogeneous_group'].agg(['count', 'mean'])

,count,mean
source,,
all2,1,1.000000
social,192,0.989583
testing,30,0.566667
toloka,1455,0.858419
toloka_selected,468,0.931624


In [20]:
words_done = df.groupby(['participant', 'word']).first().reset_index().groupby(['participant'])['word'].nunique()
# words_done.hist();

In [15]:
(words_done >= 10).mean(), (words_done >= 9).mean()

(0.6353288364249579, 0.7213322091062394)

In [16]:
words_done_fillerok = words_done[all_homogeneous.query('homogeneous_group == True')['participant']]
(words_done_fillerok >= 10).mean(), (words_done_fillerok >= 9).mean()

(0.694136291600634, 0.7908082408874801)

In [17]:
df_out = df.copy()
df_out = (df[df['participant'].isin(words_done_fillerok[words_done_fillerok >= 9].index)]
          .query('is_filler == False')
          .query("source in ['social', 'toloka', 'toloka_selected']")
          .reset_index(drop=True))
print(len(df_out))
df_out.head()

26760


,participant,source,cs,is_filler,word,group,contexts
0,277a058c-6372-4840-8b1d-2ac2bac3d5e1,social,36,False,наглый,close_close_meton,243 244
1,277a058c-6372-4840-8b1d-2ac2bac3d5e1,social,36,False,наглый,close_close_meton,245 246
2,277a058c-6372-4840-8b1d-2ac2bac3d5e1,social,36,False,наглый,close_close_meton,248 249 247
3,277a058c-6372-4840-8b1d-2ac2bac3d5e1,social,37,False,внимательный,close_close_meton,250 252
4,277a058c-6372-4840-8b1d-2ac2bac3d5e1,social,37,False,внимательный,close_close_meton,256 251 254


In [18]:
df_participants = pd.read_csv('participants.csv')
del df_participants['type']
df_participants['participant'] = df_participants.pop('person.id')
df_participants = df_participants[['participant'] + [c for c in df_participants.columns[:-1]]]
df_participants = df_participants[df_participants['participant'].isin(set(df_out['participant']))]
df_participants.head()

,participant,profession,age,leading_hand,sex,languages,education
38,277a058c-6372-4840-8b1d-2ac2bac3d5e1,Журналист,22,левая,женщина,Русский,бакалавриат
40,6d95f5d4-7b88-48f9-b239-b7db98ace329,историк,21,правая,мужчина,русский,бакалавриат
43,98f0eb92-84a5-4a90-91e7-19d5ce1f7c69,лингвист,23,левая,женщина,русский,магистратура
46,8cb2bbe8-7546-4bfd-86ab-c183ee89e5ff,филолог,22,владею обеими руками одинаково,женщина,русский,бакалавриат
50,31208ef9-ccc2-430b-8396-31f49099b749,Маркетолог,39,правая,женщина,Русский,магистратура


In [19]:
! mkdir -p out_clean
df_out.to_csv('out_clean/results.csv', index=None)
df_contexts.to_csv('out_clean/contexts.csv', index=None)
df_participants.to_csv('out_clean/participants.csv', index=None)